In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import re

Using the digits database and scikit-learn

1. Split your data into 30% test and 70% training sets, 
2. For each of the values of $C = 10^{k}$ for $k = {-10,...,10}$ train an $L^{2}$ regularized logistic regression model with regularization weight lambda = $\frac{1}{C}$  (this is the default form for scikit-learn) on the training set and compute the mean accuracy on the test set for each model.  Which performed best?  
3. Repeat #2 with $L^{1}$ regularization instead of $L^{2}$.  Do the results suggest any features that can be dropped from the data set?
4. Scikit-learn does not have logistic regression without regularization.  What values of $C$ are most similar to an un-regularized model?

In [155]:
#1.
digits = datasets.load_digits()
Xtrain, Xtest, ytrain, ytest = train_test_split(digits.data, digits.target, test_size=.3)

In [156]:
#2.
l=[]
nums=range(-10,11)
for i in range(-10,11):
    log = LogisticRegression(penalty="l2",C=10**i)
    results = log.fit(Xtrain, ytrain)
    l.append(results.score(Xtest, ytest))
print(max(l),"is the greatest score from 10**",nums[l.index(max(l))])

0.953703703704 is the greatest score from 10** -2


In [157]:
#3
l=[]
for i in range(-10,11):
    log = LogisticRegression(penalty="l1",C=10**i)
    results = log.fit(Xtrain, ytrain)
    l.append(results.score(Xtest, ytest))
print(max(l),"is the greatest score from 10**",nums[l.index(max(l))])
print("We can drop all features whose coeffs are less than our confidence interval")
print("Really big values of C are most similar to un-regularized model")

0.955555555556 is the greatest score from 10** -1
We can drop all features whose coeffs are less than our confidence interval
Really big values of C are most similar to un-regularized model


Identify a classification problem related to your final project, using your project data.  

1. Apply $L^2$ regularized logistic regression to model this with an appropriate choice of $C$ (or lambda).  Discuss how (and why) you chose your specific the value $C$.  
2. Apply $L^1$ regularized logistic regression to model this with an appropriate choice of $C$.  Discuss how (and why) you chose your specific value of $C$.  
3. Identify which features of your data to include and which to discard for a good logistic regression model for your problem.  Compare which features are suggested for removal by $L^1$ regularization (from `scikit-learn) versus using the methods we have used for linear regression, including p-values, BIC, and AIC (from statsmodels).  
Clearly identify your final preferred model, and explain why you chose that over the other contenders. 
What conclusions can be drawn from your results about the original classification question you asked?

In [139]:
data = pd.read_csv("../../../Desktop/yelpcrawl/yelp.csv", names=['name','category','rating','address','price','0'])

In [140]:
data=data[['name','category','rating','address','price']]
data=data.drop_duplicates()
data.to_csv('yelp.csv')

In [153]:
data.sample(5)

,name,category,rating,address,price
750,Las Incas,Peruvian,None,279 E 300 S StProvo UT 84606,None
457,Gandolfo's New York Deli,American(Traditional)Delis,4,715 S Main StPleasant Grove UT 84062,1
408,Sushi Garden Bistro,SushiBars,4,192 S 500th EAmerican Fork UT 84003,1
647,Chick-fil-A,FastFood,4,183 N West State RdAmerican Fork UT 84003,1
235,Molly's,American(Traditional),4.5,753 W Columbia LnProvo UT 84604,1


In [180]:
data=data[data['price']!=' None']
data=data[data['price']!='3']
data=data[data['rating']!=" None"]
Xtrain, Xtest, ytrain, ytest = train_test_split(data[['rating']], data['price'], test_size=.3)

### The purpose of this is to see if there's a correlation between restaurant price (\$ vs \$\$) and Yelp rating.

In [181]:
log = LogisticRegression(penalty="l2",C=10**-10)
results = log.fit(Xtrain, ytrain)
score = results.score(Xtest, ytest)
print("Score for L2 regression is ", score, " with C=10^-10, which was chosen by looping through like above" )

Score for L2 regression is  0.734375  with C=10^-10, which was chosen by looping through like above


In [183]:
l=[]
nums=range(-10,11)
for i in range(-10,11):
    log = LogisticRegression(penalty="l1",C=10**i)
    results = log.fit(Xtrain, ytrain)
    l.append(results.score(Xtest, ytest))
print(max(l),"is the greatest score from 10**",nums[l.index(max(l))])
print("The same value of C was used in this regression. After running the code several times, I've determined L1 vs L2 makes no difference on this data")

0.734375 is the greatest score from 10** -10
The same value of C was used in this regression. After running the code several times, I've determined L1 vs L2 makes no difference on this data


This dataset was a little bit difficult to work with because most of what I want to predict is dependent on the category tags, so there's less numerical data to work with. Hence only rating and price were compared. That could be why L1 and L2 gave the same answers each time.
I imagine since this was only contingent on one variable that none of the models would suggest removing it. I prefer logistic regression because we spent more than 5 minutes on it in class so I feel like I understand it more. Maybe that's a bad reason.....

After running the regression, we can determine that there is a correlation between price and perceived quality of a restaurant, but it doesn't hold true all the time. People prefer expensive things, but McDonald's is still pretty great.

In [ ]:
def format_categories(string):
    